In [2]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=ed1bb12d180b394b52a2da224272b765efd9e63a8363e597001960db1c049744
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [3]:
pip install face_recognition

     |████████████████████████████████| 100.2MB 89kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=8f49b84cb523ef6d50b9347a7e126f31027b785e58181bebec9dda737df735b9
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [4]:
import os
import time as tm
import cv2
import pickle
import dlib
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

import wikipedia as wiki
import pandas as pd

GENRE_TYPES = 'Talk-Show|News'
TITLE_TYPE = 'tvSeries'
PROFESSION_TYPES =  [' host',' presenter',' journalist',' news anchor','correspondent',' anchor']
SPLIT_TYPES = [' is a ',' is an ',' was a ',' was an ',' is the ']
# IMDB_PROFESSION_TYPES = 'actor|actress|miscellaneous|Null'
NONE = 'Not Found'
PAGE_ERROR = 'does not match any pages'
DISAMBIGUATION_ERROR = 'disambiguation'
NETWORK_TYPES = ['FOX','ABC','CBS','NBC','CNN','United Paramount Net','Warner Bros.','Pure Independent','PBS','Pax TV','Telemundo']
REGION_TYPE = 'US'
SHOW_NOT_IN_IMDB = 'Show is not present in IMDb'
NAME_BASICS = "/content/drive/MyDrive/Datasets/IMDB_Datasets/name.basics.tsv/data.tsv"
TITLE_BASICS = "/content/drive/MyDrive/Datasets/IMDB_Datasets/title.basics.tsv/data.tsv"
TITLE_PRINCIPALS = "/content/drive/MyDrive/Datasets/IMDB_Datasets/title.principals.tsv.gz"
TITLE_AKAS = "/content/drive/MyDrive/Datasets/IMDB_Datasets/title.akas.tsv/data.tsv"
TITLE_CREW = "/content/drive/MyDrive/Datasets/IMDB_Datasets/title.crew.tsv/data.tsv"

In [6]:
def loadPickle(filename):
    infile = open(filename,'rb')
    return pickle.load(infile, encoding='latin1')

def storePickle(filename, data):
    file = open(filename,'wb')
    pickle.dump(data, file)
    file.close()


def sec2HMS(seconds):
    return tm.strftime('%H:%M:%S', tm.gmtime(seconds))

def HMS2sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

celebs, celeb_encodings = loadPickle('/content/drive/MyDrive/Datasets/Show_Segmentation_2020/final_celeb_detection/final_pickles/anchors-with-TV-encodings.pickle')
celeb_encodings = np.array([np.array(x) for x in celeb_encodings])


# Populating KNN space with labelled encodings
X = []
Y = []
for i in range(len(celeb_encodings)): #prepare dataset
    for celeb_encoding in celeb_encodings[i]:
        X.append(celeb_encoding)
        Y.append(celebs[i])
        
neigh = KNeighborsClassifier(n_neighbors=30)
neigh.fit(X, Y)


def encoding2name(f_encodings):
        return neigh.predict(f_encodings)


vid_path = '/content/drive/MyDrive/Datasets/2006-01-02_0000_US_00001057_V11_M2_VHS10_H4_JA.mp4'


#TODO - take input from the user - next cell as well
output_path = '/content/drive/My Drive/Datasets/newtest'

if not os.path.exists(output_path):
    os.mkdir(output_path)


class Show:
    def __init__(self, hosts, start_time, end_time):
        self.hosts = hosts
        self.start_time = start_time
        self.end_time = end_time


def getEncodings(vid_path, skip_seconds=1):
    """ Returns 129D encodings for each face present in the video
        First 128 are the face encodings and the last value is the time.
        
       :param interval: (in seconds) frame interval to skip and look for faces
       :param model: 'hog' is less accurate but faster compared to 'cnn'
       :param store: if True, stores the faces in directory specified by dirName """
    vidcap = cv2.VideoCapture(vid_path)
    cv2.VideoCapture 
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    interval = int(fps+1)*skip_seconds #no. of frames to skip
    print("FPS of the video: {}".format(fps))
    allEncodings = [] #Dict containing path, box and encoding
    n_frame = 0 # no. of frame being processed

    success, frame = vidcap.read()
    
    while success:
        rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            
        bboxes = face_recognition.face_locations(rgb,model='hog')
        encodings = face_recognition.face_encodings(rgb,bboxes)
        for i,bbox in enumerate(bboxes): #for each found face in the frame
            top,right,bottom,left = bbox[0],bbox[1],bbox[2],bbox[3]
            face_img = frame[top:bottom, left:right]
            
            d = {'time': (n_frame/fps), 'loc': bbox, 'encoding':encodings[i]}
            # print(d)
            allEncodings.append(d)
            
        n_frame += interval            
        vidcap.set(cv2.CAP_PROP_POS_FRAMES,n_frame)
        success, frame = vidcap.read()
    
    cv2.destroyAllWindows()
    vidcap.release()
    return allEncodings


def clusterFaces(allEncodings, n_jobs=-1, max_clusters = 100):
    """face_times: a Tuple list with x[0] as face_classes, and x[1] as list of their
    times of occurence in the video.
    
       face_encodings: dict mapping face_class to list of encodings of all occurences 
       of that face in the video"""
    encodings = [d['encoding'] for d in allEncodings]
    times = [d['time'] for d in allEncodings]
    # loc = [d['loc'] for d in allEncodings]
    # file_names = [d['time'] for d in allEncodings]
    clt = DBSCAN(metric='euclidean', n_jobs=n_jobs, min_samples=5,eps=0.37)
    clt.fit(encodings)
    labels = clt.labels_ #-1 => too small to include in a cluster
    
    face_times = [] #list of tuples, first element is time of occurence, 2nd is the class of the face
    face_encodings = {} #dict mapping from a face_classs to all encodings of that faces (from the images found in the video)
    # face_index = []

    for i,label in enumerate(labels):
        encoding = encodings[i]
        time = times[i]
        if(label!=-1):
            face_times.append((int(time), str(label)))
            
            if label in face_encodings:
                face_encodings[label].append(encoding)
            else:
                face_encodings[label] = [encoding]
    return face_times,face_encodings


def addEmptyFaces(faces, skip_seconds):
    """Modifies faces dict to include timestamps where no faces are present
       '-1' is the value assigned to these.
       :skip_gap: 'interval' parameter given in file2encoding() function (in seconds)"""
    min_time = (faces[0][0])
    max_time = (faces[-1][0])
    curr_time = min_time
    faces_empty = []
    counter = 0
    
    while (curr_time < max_time):
        if((faces[counter][0]) > curr_time): #No face found at this time
            faces_empty.append(((curr_time), '-1'))
        else:                              #Face was already marked at this time
            faces_empty.append(faces[counter])
            counter+=1
        curr_time += skip_seconds
    return faces_empty


def faceTrendsDuration(faces, interval = 900, overlapping = False, join_consecutive = False,n_top=10):
    """Trendy faces are the faces of an actor which occur the most in a given interval.
       Video is split into *interval*s and most occuring faces in them are noted.
       For each interval, *n_top* no. of most occuring faces are returned
       in a dict format."""
    #GOTO JUMPER if change interval
    # interval - SKIP_INTERVAL*interval time duration is taken as length of one trend_bucket
    trending_face = faces[0][1] #First face's class
    trendy_faces = {}
    
    if overlapping:
        skip=1
    else:
        skip=interval
    for x in range(0, len(faces), skip):
        face_count = {} #Keeps count of no. of instances of each face_class
        interval_string = sec2HMS(faces[x][0])
        for face in faces[x:min(len(faces),x+interval)]:
            curr_time = face[0]
            curr_face = face[1]
            if curr_face == '-1':
                continue
                
            if curr_face in face_count:
                face_count[curr_face] = (face_count[curr_face][0],curr_time)
            else:
                face_count[curr_face] = (curr_time, curr_time)
                
        if face_count: # if face_count is not empty
            max_face_in_interval = sorted(list(face_count.keys()), key =(lambda key: (face_count[key][1]) - (face_count[key][0])),reverse=True)[:n_top]
        else:
            max_face_in_interval = ['-1']
        if join_consecutive:
            if(max_face_in_interval!=trending_face):
                trending_face = max_face_in_interval
                trendy_faces[interval_string] = trending_face
        else:
            trending_face = max_face_in_interval
            trendy_faces[interval_string] = trending_face

    return trendy_faces


SKIP_SECONDS = 1 
allEncodings = getEncodings(vid_path, SKIP_SECONDS)
face_list, face_encodings = clusterFaces(allEncodings)


faces_empty = addEmptyFaces(face_list, SKIP_SECONDS)
trends = faceTrendsDuration(faces_empty)


face_dict = {} #dict having all occurences of each face
for x in face_list: 
    if x[1] in face_dict:
        face_dict[x[1]].append(x[0])
    else:
        face_dict[x[1]] = [x[0]]


#Getting consecutives
cons_dict = {}
for key,vals in trends.items():
    key = HMS2sec(key)
    for val in vals:
        if val in cons_dict:
            if (cons_dict[val][-1][-1]==prev_time):
                cons_dict[val][-1].append(key)
            else:
                cons_dict[val].append([key])
        else:
            cons_dict[val] = [[key]]
    prev_time = key


face_intervals = {} #Dict containing exact timestamps of all occurences of an actor's face
                    #in intervals specified by 'cons_dict'
    
for face,intervals in cons_dict.items():
    face_intervals[face] = []
    for times in intervals:
        lb = min(x for x in face_dict[face] if x >= times[0]) #lower bound
        ub = max(x for x in face_dict[face] if (x <= times[-1]+900)) #upper bound
        face_intervals[face].append([x for x in face_dict[face] if (x>=lb and x<=ub)])


#Converting the dict to a bunch of tuples of the form (face,each_interval)
shows = [(face,times) for face in face_intervals.keys() for times in face_intervals[face]]
shows = sorted(shows, key = lambda x: x[1][-1]) #Sorting face intervals by their order of ending time.
shows = [list(x) for x in shows]


#Removing too short
min_len = 0.5*60 #In seconds #CHANGE
shows = [x for x in shows if (x[1][-1] - x[1][0])>=min_len]
#Will still be sorted by ending time


# Removing intervals within intervals:
show_intervals = [x[1] for x in shows]
i = 0
for x in range(len(shows)):
    curr_interval = shows[i][1]
    for x in show_intervals:
        if(curr_interval[0]>x[0] and curr_interval[-1]<x[-1]):
            del(shows[i])
            i -= 1
            break
    i += 1   


# Combining consecutive shows with very high overlap
i = 0
overlap_threshold = 0.75
while(i<len(shows)-1):
    diff = shows[i][1][-1] - shows[i+1][1][0]
    #total = shows[i+1][1][-1] - shows[i][1][0]
    short_show = min(shows[i][1][-1]-shows[i][1][0],shows[i+1][1][-1]-shows[i+1][1][0])
    overlap = diff/short_show
    if(overlap > overlap_threshold):
        print('Hosts: {} & {}'.format(shows[i][0],shows[i+1][0]))
        print('Original durations: {} to {} and {} to {}'.format(sec2HMS(shows[i][1][0]),sec2HMS(shows[i][1][-1]),sec2HMS(shows[i+1][1][0]),sec2HMS(shows[i+1][1][-1])))
        print('Total duration: '+str(diff))
        print('Overlap: '+str(overlap))
        lb = min(shows[i][1][0],shows[i+1][1][0])
        ub = max(shows[i][1][-1],shows[i+1][1][-1])
        shows[i][0] = shows[i][0]+'&'+shows[i+1][0]
        shows[i][1].extend(shows[i+1][1])
        shows[i][1] = sorted(shows[i][1])
        print('Merging show {} from {} to {}'.format(shows[i+1][0],shows[i+1][1][0],shows[i+1][1][-1]))
        print()
        del(shows[i+1])
    else:
        i += 1


#Removing intervals which are overlapping between two shows.
#Example: A - 01:00 to 10:00
#         B - 09:00 to 12:00
#         C - 10:00 to 20:00
DOUBLE_OVERLAP_THRESHOLD = 0.85
i=1
while (i<len(shows)-1):
    curr_show = len(shows[i][1]) #Length of current show
    diff1 = len([x for x in shows[i][1] if x in range(shows[i-1][1][0],shows[i-1][1][-1])]) #Left side overlapping
    overlap1 = diff1/curr_show
    diff2 = len([x for x in shows[i][1] if x in range(shows[i+1][1][0],shows[i+1][1][-1])]) #Right side overlapping
    overlap2 = diff2/curr_show
    net_overlap = overlap1 + overlap2

#actual algorithm
    if(net_overlap > DOUBLE_OVERLAP_THRESHOLD):
        print('Hosts: {} and {} and {}'.format(shows[i-1][0],shows[i][0],shows[i+1][0]))
        print('Original durations: {} to {} and {} to {} and {} to {}'.format(sec2HMS(shows[i-1][1][0]),sec2HMS(shows[i-1][1][-1]),sec2HMS(shows[i][1][0]),sec2HMS(shows[i][1][-1]),sec2HMS(shows[i+1][1][0]),sec2HMS(shows[i+1][1][-1])))
        #         print('Total duration: '+sec2HMS(diff))
        #         print('Overlap: '+str(overlap))
        print('Left overlap: {}'.format(overlap1))
        print('Right overlap: {}'.format(overlap2))
        print('Net overlap: {}'.format(net_overlap))
        print()
        del(shows[i])
    else:
        i+=1


shows_refined = [shows[0]]
prev_show = shows[0]
for show in shows[1:]:
    shows_refined.append([show[0], [x for x in show[1] if x>=prev_show[1][-1]] ])
    prev_show = show


shows = [Show(str(x[0]),x[1][0],x[1][-1]) for x in shows_refined]


for show in shows:
    hosts = show.hosts.split('&') #getting list of hosts of the show
    hosts = sorted(hosts, key = lambda x: len(face_encodings[int(x)]), reverse=True) #Most occuring anchor is taken as the main anchor
    for i in range(len(hosts)):
        host = hosts[i]
        host_encodings = face_encodings[int(host)]
        # print(host_encodings)      #Getting all encodings of this host's face
        host_prob_names = Counter(list(encoding2name(host_encodings)))
        # print(host_prob_names) #Getting predictions of all faces
        hosts[i] = [(x,y/len(host_encodings)) for x,y in host_prob_names.most_common()] #sorting the predictions by their frequency
        # print(hosts[i])
    show.hosts = hosts
    print(show.hosts)
#     show.hosts = sorted(hosts, key = lambda x: sum(y for _,y in x))
# (neigh.kneighbors(face_encodings[0]))[1]


#Extracting video metadata
vcap = cv2.VideoCapture(vid_path)
vid_width, vid_height = int(vcap.get(3)), int(vcap.get(4))
vcap.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
vid_duration = int(vcap.get(cv2.CAP_PROP_POS_MSEC)/1000)
cv2.destroyAllWindows()
vcap.release()


filename = os.path.splitext(os.path.basename(vid_path))[0]
attributes = filename.split('_')
pulldate, barcode = attributes[0], attributes[3]
vid_txt3_path = os.path.splitext(vid_path)[0]+'.txt3'
txt3_subtitles = None

#common headers for all cuts - default values (to be used if this column is not present in the txt3)
OVD = 'OVD|'+filename+'.mp4'  
OID = 'OID|'
COL = 'COL|Communication Studies Archive, UCLA'
SRC = 'SRC|Rosenthal Collection, UCLA'
LAN = 'LAN|ENG'
LBT = 'LBT|'

if os.path.exists(vid_txt3_path):
    txt3_lines = open(vid_txt3_path, 'r').read().splitlines()
    
    for i in range(len(txt3_lines)):
        if txt3_lines[i][3]!='|': #Header lines end
            txt3_headers = txt3_lines[:i]
            txt3_subtitles = txt3_lines[i:] #Subtitles' lines start here
            break
            
    for header in txt3_headers:
        if header[:3]=='TOP':
            OVD = 'OVD|'+header[4:]
        elif header[:3]=='UID':
            OID = 'OID|'+header[4:]
        elif header[:3]=='COL':
            COL = header
        elif header[:3]=='SRC':
            SRC = header
        elif header[:3]=='LAN':
            LAN = header


#Cutting shows from the main video + making a .txt file for each
for n_show, show in enumerate(shows):
    
    channel = 'unknown-channel' #until the work with IMDb is done
    channel = channel.replace(' ', '_')
    
    main_host = show.hosts[0][0]
    if main_host[1] > 0.45: #If majority predictions are of the same person
        host_name = main_host[0]
    else:
        host_name = 'unknown-host'
#     host_name = show.hosts[0][0][0]
    host_name = host_name.replace(' ', '_')
    cut_filename = '_'.join((pulldate, barcode, '-'.join((str(n_show+1), str(len(shows)))), channel, host_name))
    cut_path = os.path.join(output_path, cut_filename)
    cut_starttime = (int(max(0, show.start_time - 60))) #using a buffer of 1 minute
                            
    if n_show==len(shows)-1: #last show                            
        cut_endtime = (int(min(show.end_time + 60, vid_duration)))
    else:
        cut_endtime = (int(shows[n_show+1].start_time)) #till the start of next shows
    
    cut_duration = sec2HMS(cut_endtime - cut_starttime)
    cut_starttime = sec2HMS(cut_starttime)
    
    ffmpeg_command = 'ffmpeg -ss {} -t {} -i {} -vcodec copy -acodec copy {}.mp4'.format(cut_starttime, cut_duration, vid_path, cut_path)
    os.system(ffmpeg_command)
    
    TOP = 'TOP|'+cut_filename+'.mp4'
    UID = 'UID|' #TODO: Generate UUID in the cluster
    TTL = 'TTL|'
    PID = 'PID|'
    CMT = 'CMT|'
    INF = 'INF|'
    for i, host in enumerate(show.hosts):
        INF += 'probable_host'+str(i+1)+':'+'_'.join([pred[0].replace(' ','-') for pred in host][:5])+'_'
    INF = INF[:-1]        
    DUR = 'DUR|'+cut_duration
    TMS = 'TMS|'+cut_starttime+'-'+sec2HMS(cut_endtime)
    VID = 'VID|{}x{}'.format(vid_width, vid_height)
    
    #initializing with headers
    cut_txt_lines = [TOP, COL, UID, SRC, TTL, PID, CMT, DUR, VID, LAN, LBT, OVD, OID, TMS, INF] 
    
    sub_starttime = pulldate.replace('-','') + cut_starttime.replace(':','')
    sub_endtime = pulldate.replace('-','') + sec2HMS(cut_endtime).replace(':','')
    
    
    if txt3_subtitles:
        for sub_idx in range(len(txt3_subtitles)): #TODO: maybe make the starting time 0 for each?
            curr_sub = txt3_subtitles[sub_idx]
            
            if curr_sub[:14] >= sub_starttime:
                if curr_sub[:14] <= sub_endtime:
                    cut_txt_lines.append(txt3_subtitles[sub_idx])
                else:
                    break

    with open(cut_path+'.txt', 'w') as f:
        for line in cut_txt_lines:
            f.write("%s\n" % line)

ModuleNotFoundError: ignored

In [6]:
""" After getting all professions of a person we can easily confirm that
    our previous top 5 prediction is right or not & by using this we find 
    correct anchor.
"""       

def get_wiki_professions(Anchor_Name):
  if len(wiki.search(Anchor_Name)) == 0:
    return NONE

  Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
  Store_name = Read_Namebasics.primaryName.tolist()

  try:
    if Anchor_Name in Store_name:
      anchor_content = (((wiki.page(Anchor_Name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
    else:
      return NONE
  except wiki.exceptions.PageError as p:
    if PAGE_ERROR in str(p):
      return NONE
  except wiki.exceptions.DisambiguationError as e:
    Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
    Store_name = Read_Namebasics.primaryName.tolist()
    if Anchor_Name in Store_name:
      # print(e.options)
      list_disambiguation_error = e.options
      for i in list_disambiguation_error:
        if i.find(DISAMBIGUATION_ERROR)!= -1:
          return NONE
        # print(i)
        anchor_content = (((wiki.page(i,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
        # print(anchor_content)
        
        split_type = SPLIT_TYPES

        split_type_list = []
        prefix = None
        for x in split_type:
          if x in anchor_content:
            prefix = x
            split_type_index = anchor_content.find(prefix)
            #  print(index_split_type)
            pro = split_type_list.append(split_type_index)
          else:
            split_type_list.append(1000)

        low_index = split_type[split_type_list.index(min(split_type_list))]
        #  print(low_index)   
            

        if prefix is None:
          return NONE

        final_split = anchor_content.split(low_index)[1].split('. ')[0].split(', ')
        # print(final_split)
        final_professions = [anchor_content.lower() for anchor_content in final_split]
        # return final_professions
        # print(final_professions)
        required_professions = PROFESSION_TYPES

        check_individual_prof = []
        for x in required_professions:
          final_check= [(x in y) for y in final_professions]
          check_individual_prof.append(any(final_check))
          # print(x)
        
        if (any(check_individual_prof)==True):
          return final_professions
      else:
        return NONE
    
    else:
      return NONE     

  # anchor_content = (((wiki.page(name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
  #  print(anchor_content)
  split_type = SPLIT_TYPES
  
  split_type_list = []
  prefix = None
  for x in split_type:
    if x in anchor_content:
      prefix = x
      split_type_index = anchor_content.find(prefix)
      #  print(index_split_type)
      pro = split_type_list.append(split_type_index)
    else:
      split_type_list.append(1000)

  low_index = split_type[split_type_list.index(min(split_type_list))]
  #  print(low_index)   
      

  if prefix is None:
    return NONE

  final_split = anchor_content.split(low_index)[1].split('. ')[0].split(', ')
  # print(final_split)
  final_professions = [anchor_content.lower() for anchor_content in final_split]
  return final_professions


def check_wiki_professions(list_profession):
  required_professions = PROFESSION_TYPES

  check_individual_prof = []
  for x in required_professions:
    final_check= [(x in y) for y in list_profession]
    check_individual_prof.append(any(final_check))
    # print(final_check)
    # print(x)
  
  if (any(check_individual_prof)==True):
    # print(check_individual_prof)
    return True
  else:
    return False


"""  From top 5 anchor we get most probable anchor in that show
     Make sure that the input anchors list follows given format. Here's an example
     Anchor_list = "Colleen-Williams_Ashwini-Bhave_Elaine-Quijano_Becky-Hobbs_Heidi-Collins"
"""
def get_correct_anchor(top_5):
  top_5_list = (top_5.replace('-',' ').replace('_',',')).split(',')
  # print(top_5_list)
  top_5_list = list(filter(None, top_5_list))
  for x in top_5_list:
    # print(x)
    # if Anchor_Name in Store_name:
    # pqe = check_wiki_professions(get_wiki_professions(x)) is True:

    if check_wiki_professions(get_wiki_professions(x)) is True:
      Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
      Store_name = Read_Namebasics.primaryName.tolist()
      if x in Store_name:
        return x 

    # if pqe is True:
      # list_of_selected_anchor.append(x)
     
  return ''


## get finals anchors from the multiple probable hosts

def Final_Anchor(txt_path):
  filter_probable_list = []
  all_lines = []                              
  with open (txt_path) as myfile:  
      for line in myfile:                   
          all_lines.append(line)
  # print(len(all_lines))                      
  last_line = all_lines[len(all_lines)-1].split('INF|')[1].split('\n')[0].replace(' ','')
  # print(last_line)
  avoid_num = ''.join([i for i in last_line if not i.isdigit()])
  probable_host_list = avoid_num.split('probable_host:')
  probable_host_list = list(filter(None, probable_host_list))

  # print(probable_host_list)

  for i in probable_host_list:
    filter_probable_list.append(get_correct_anchor(i))
    filter_probable_list = list(filter(None, filter_probable_list))

  confident_host_line = 'Confident Host: '
  for i in filter_probable_list:
    confident_host_line += str(i) +' |';

  with open(txt_path, "a+") as file_object:
    file_object.seek(0)
    data = file_object.read(100)
    if len(data) > 0 :
        file_object.write("\n")
    file_object.write(confident_host_line)  
  return filter_probable_list


#here we are getting all anchors present in a video
def get_list_anchors(folder_path):#folder path = where all txt files placed of a show 
  list_of_anchors = []
  list_txt_files = os.listdir(folder_path)
  for i in list_txt_files:
    txt_file_path = folder_path+'/'+i
    # print(Final_Anchor(txt_file_path))
    list_of_anchors.append(Final_Anchor(txt_file_path))
    print("Wait it will take 10-15 min")
  return list_of_anchors


#this will give us network name either from anchor or show
def get_channel(name):
  try:
    anchor_content = ((((wiki.page(name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. ')))
  
  except wiki.exceptions.PageError as p:
    if PAGE_ERROR in str(p):
      return NONE
  
  except wiki.exceptions.DisambiguationError as e:
    list_disambiguation_error = e.options
    for anchor_name in list_disambiguation_error:
      if anchor_name.find(DISAMBIGUATION_ERROR)!= -1:
        return NONE
      
      else:
        anchor_content = ((((wiki.page(anchor_name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. ')))
  # print(anchor_content)
  Networks_list = NETWORK_TYPES
  Network_filter = []
    
  for network in Networks_list:
    # print(word)
    if anchor_content.find(network)!=-1:
      Network_filter.append(network)
  return Network_filter


def get_majority_network(anchors_list):
  list_of_network = []
  for anchorlist in anchors_list:
    for anchor in anchorlist:
      # print(a)
      list_of_network.append(get_channel(anchor))
  list_of_network = list(filter(None,list_of_network))
  print(list_of_network)

  X ={} #dict contains networks and respected votes 
  for network in list_of_network:
    for particular in network:
      if particular in X:
        X[particular] +=1
      else:
        X[particular] = 1
  print(X)   
  Max_votes = max(X.values())
  # print(Max_votes)
  Majority_Network = [particular for particular, votes in X.items() if votes == Max_votes]
  return Majority_Network


def get_nconst(Anchor_Name,Pull_Year):

  Read_Titlebasics = pd.read_table(TITLE_BASICS, sep='\t')
  Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
  
  Read_Namebasics.primaryProfession = Read_Namebasics.primaryProfession.fillna("Null")
  filter_year = Read_Namebasics[Read_Namebasics.deathYear >= Pull_Year]
  Search_name = filter_year[filter_year.primaryName == str(Anchor_Name)]

  if len(Search_name.index) == 1:
    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
      return [row.nconst,prepare_list_kft]

  else:
     
    titleakas = pd.read_table(TITLE_AKAS, sep='\t')
    titleakas.region = titleakas.region.fillna(REGION_TYPE)
    filter_region = titleakas[titleakas.region == REGION_TYPE]
    filter_region_titleid = filter_region.titleId.tolist()

    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
      for i in prepare_list_kft:
        take_title = Read_Titlebasics[Read_Titlebasics.tconst == i]
        if i in filter_region_titleid:
          check_genres = take_title[take_title.genres.str.contains(GENRE_TYPES)]
          if len(check_genres.index) == 1:
            return [row.nconst,prepare_list_kft]
    return [NONE]



def get_all_shows(nConst):
  chunksize = 6*10**6
  titleprincipal = pd.read_table(TITLE_PRINCIPALS, chunksize = chunksize)
  all_shows = pd.DataFrame(columns=['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'])
  #Initializing with the same columns as title.principals
  for i,chunk in enumerate((titleprincipal)):
    req_shows = chunk[chunk['nconst'] == nConst]
    print(' {} shows in chunk {}'.format(len(req_shows), i))
    all_shows = all_shows.append(req_shows, ignore_index = True)
    # filter_category = all_shows[all_shows.category == 'self']

  return all_shows.tconst.tolist()


# get final titles afters union or filter
def get_show_names(Anchor_Name,Pull_Year):
  store_US_titles = []

  store_nconst = get_nconst(Anchor_Name,Pull_Year)

  if store_nconst[0] == NONE:
    return None

  titlecrew = pd.read_table('/content/drive/MyDrive/Datasets/IMDB_Datasets/title.crew.tsv/data.tsv' ,sep='\t')
  nconst_in_dir = titlecrew[titlecrew.directors.str.contains(store_nconst[0])]
  nconst_in_writer = titlecrew[titlecrew.writers.str.contains(store_nconst[0])]
  tconst_in_crew = list(set().union(nconst_in_dir.tconst.tolist(),nconst_in_writer.tconst.tolist()))

  # Combine_list_1 = list(set().union(store_nconst[1],get_all_shows(store_nconst[0])))
  Combine_list_2 = set().union(store_nconst[1],get_all_shows(store_nconst[0]),tconst_in_crew)

# Here applying US(region) filter in the shows

  titleakas = pd.read_table(TITLE_AKAS, sep='\t')
  titleakas.region = titleakas.region.fillna(REGION_TYPE)
  filter_region = titleakas[titleakas.region == REGION_TYPE]
  filter_region_titleid = filter_region.titleId.tolist()
  for x in Combine_list_2:
    if x in filter_region_titleid:
      store_US_titles.append(x)

  # return store_US_titles
  tConsts= set(store_US_titles)

  chunksize = 6*10**6

  titlebasics = pd.read_table(TITLE_BASICS, chunksize = chunksize)
  Show_Names = pd.DataFrame(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
  for i,chunk in enumerate(titlebasics):
    show_names = chunk[chunk['tconst'].isin(tConsts)]
    print('{} shows in chunk {}'.format(len(show_names), i))
    Show_Names = Show_Names.append(show_names, ignore_index = True)
    # print(Show_Names)
    filter_title_type = Show_Names[Show_Names['titleType']==TITLE_TYPE]
    filter_genres = filter_title_type[filter_title_type.genres.str.contains(GENRE_TYPES)]
    filter_end_year = filter_genres[filter_genres.endYear >= Pull_Year]
    filter_start_year = filter_end_year[filter_end_year.startYear.apply(str) <=Pull_Year]
  return filter_start_year


def final_show(anchor_list, Pull_Year):
  X = {} #dict contains show name and their votes
  for host in anchor_list:
    print(host)
    store_show_names = get_show_names(host,Pull_Year)
    if store_show_names is None:
      title = SHOW_NOT_IN_IMDB
      if title in X:
        X[title] +=1
      else:
        X[title] = 1
    
    else:
      for title in store_show_names.primaryTitle.tolist():
        if title in X:
          X[title] +=1
        else:
          X[title] = 1
  if bool(X) is False:
    return NONE
  else:
    Max_votes = max(X.values())
    # print(Max_votes)
    Majority_Shows = [title for title, votes in X.items() if votes==Max_votes]
    if len(Majority_Shows) is 1:
      print(Majority_Shows)
      # return Majority_Shows
    else:
      Open_Subtitles = open("/content/drive/MyDrive/Datasets/Subtitles/2006-01-04_0000_US_00000063_V2_MB7_VHS7_H11_MS.txt3")
      Read_Subtitles = Open_Subtitles.read()

      for show in Majority_Shows:
        if Read_Subtitles.find(show.upper())!=-1:
          print(show)
          return show
          

      for show in Majority_Shows:
        for network in get_majority_network(anchor_list):
          if show.find(network)!=-1:
            print(show)
            return show




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
final_show(get_list_anchors("/content/drive/MyDrive/Datasets/newtest"),"2006")

Wait it will take 10-15 min
Wait it will take 10-15 min
['Rosemary Church', 'Boy Abunda']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:339: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:339: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


['Kyra Phillips', 'Reggie Yates']
['Show is not present in IMDb']
